# Nearest Neighbors Algorithms

The basic concept for **Nearest Neighbors Algorithms** (NNAs) is to classify a datum by finding one or more points that have similar features. The most similar points are called the **nearest neighbors**. Once found, the input datum can be put in the same class as its neighbors. We can also use this to predict the value of missing features for the input datum.

## k-Nearest Neighbors

The most commonly used NNA is **k-Nearest Neighbors,** in which the top $k$ nearest neighbors (best matches) are identified. In most instantiations of k-NNA, classification or prediction is based on a **majority vote** of the $k$ nearest neighbors.
>For example, if $k = 5$, and at least 3 out of the 5 nearest neighbors to an input datum are class A, then we would assign the new datum to class A.

For a more complex example, see the image below. Here, if $k = 1$, the green circle would be assigned to Class 1, since the nearest point is a blue square. However, if $k = 3$ the answer becomes Class 2, since the next two closest are both red triangles.


![alt text](https://cdn-images-1.medium.com/max/1600/0*Sk18h9op6uK9EpT8.)

## Quick example to illustrate a kNN:

In this section, we will use sklearn to build a k-NNA model for a data set describing cars. Our goal will be to classify the cars into one of two categories: "cool" or "uncool." Clearly these are subjective terms, but that's ok: we will provide a manually classified training set.

For example, if we consider the variables *horsepower, number of seats,* and *manual (0) or automatic (1)*, our manually classified training set might look like this:

*   150, 5, 0, uncool (2008 Honda Civic)  
*   320, 5, 0, cool (2011 Dodge Charger)
*   383, 3, 1, cool (1985 Chevy Blazer)
*   210, 7, 0, uncool (2001 Honda Odyssey)

Let's say we're trying to predict whether the 2017 Bugatti Veyron (1500hp, 2 seats, manual: 1) is cool or not. Our first step is to load the data into a python structure.

In [1]:
# -- imports -- #
import numpy as np
import pandas as po

# -- loading the data -- #
cars_dict = {'2008 Honda Civic':    {'hp':150., 'seats':5., 'auto':0., 'cool':0}, 
             '2011 Dodge Charger' : {'hp':320., 'seats':5., 'auto':0., 'cool':1}, 
             '1985 Chevy Blazer':   {'hp':383., 'seats':3., 'auto':1., 'cool':1}, 
             '2001 Honda Odyssey':  {'hp':210., 'seats':7., 'auto':0., 'cool':0}, 
             '2017 Bugatti Veyron': {'hp':1500.,'seats':2., 'auto':1., 'cool':None}}

data = po.DataFrame.from_dict(cars_dict,orient='index')
data

,hp,seats,auto,cool
1985 Chevy Blazer,383.0,3.0,1.0,1.0
2001 Honda Odyssey,210.0,7.0,0.0,0.0
2008 Honda Civic,150.0,5.0,0.0,0.0
2011 Dodge Charger,320.0,5.0,0.0,1.0
2017 Bugatti Veyron,1500.0,2.0,1.0,NaN


### Normalizing Data and Calculating Distance

To determine the class of our Bugatti using NNA, we need to measure its "nearness" to other cars. One of the simplest metrics for this is Euclidian distance, defined as the square root of the sum of the difference between each feature squared:

![alt text](https://i.stack.imgur.com/2y0bx.png.)

Some other approaches include Chi square distance and cosine distance. For further reading on distance functions, see this article: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4978658/.

Note that **by using Euclidian distance, we are normalizing the data in a way that assumes all features are equally important**. One consequence of this is that values that are generally larger (such as horsepower) will end up having a larger impact on the distance. Unless we have some reason to believe that the larger features are more important, we will want to balance features by **normalizing the data.**


One quick and easy way to normalize data is to divide each datum by the maximum value in its category. Let's do that for our data set:

In [2]:
# Normalizing the data by dividing each value by the maximum value in its row. 
# Remember that we don't normalize the label ()

data_norm_divide = data

for i in ['hp','seats','auto']:
    data_norm_divide[i] = data[i]/max(data[i].values)
    
data_norm_divide

,hp,seats,auto,cool
1985 Chevy Blazer,0.255333,0.428571,1.0,1.0
2001 Honda Odyssey,0.140000,1.000000,0.0,0.0
2008 Honda Civic,0.100000,0.714286,0.0,0.0
2011 Dodge Charger,0.213333,0.714286,0.0,1.0
2017 Bugatti Veyron,1.000000,0.285714,1.0,NaN


As usual, a tool already exists to do this. Using sklearn, we can normalize data with an object called a StandardScaler. We will use this in the example below, but feel free to also read the documentation here: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

Normalization, or more generally "standardization," is a common requirement for machine learning estimators. Beyond shifting values to similar magnitudes, many estimators will run into trouble if the values within individual features are not roughly normally distributed. By **normal distribution,** we mean a Gaussian distribution with 0 mean and unit variance. The sklearn StandardScaler will attempt to scale all the data to approximate a normal distribution for us.

In [3]:
# Normalize data by removing the mean and scaling to unit variance from each feature

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_unitnorm = po.DataFrame.from_dict(cars_dict,orient='index')

for i in ['hp','seats','auto']:
    feature_data = data_unitnorm[i].values.reshape(-1, 1)
    scaler.fit(feature_data)
    data_unitnorm[i] = scaler.transform(feature_data)
    
data_unitnorm

,hp,seats,auto,cool
1985 Chevy Blazer,-0.259004,-0.802955,1.224745,1.0
2001 Honda Odyssey,-0.604742,1.491202,-0.816497,0.0
2008 Honda Civic,-0.724651,0.344124,-0.816497,0.0
2011 Dodge Charger,-0.384908,0.344124,-0.816497,1.0
2017 Bugatti Veyron,1.973305,-1.376494,1.224745,NaN


Now that the data is standardized, we can begin building our NNA algorithm. First, we will need a function that **calculates the Euclidean distance between two data points**. For our purposes, we will assume the data points will be stored in arrays of the same length.

In [4]:
# Distance function using formula for euclidean distance

def euclidean_dist(datum1, datum2):
    inner_val = 0.0
    
    for g in range(datum1.shape[0]):
        inner_val += (datum1[g]- datum2[g]) ** 2
    
    distance = math.sqrt(inner_val)
    return(distance)

Next, we calculate the distance between our Bugatti and each other car using the *euclidean_dist* function. For the sake of testing, we do this twice: once for the normalized data (*data_norm_divide*), and once for the data standardized by sklearn (*data_unitnorm*). Note that we will not input the classification column of cool/uncool to our distance function, which means taking a subset of each array.

In [5]:
# FYI: This is how you can call a specific row by name and sub-select features
bugatti = data.loc["2017 Bugatti Veyron"][["hp","seats","auto"]].values

import math

# -- normalized data by dividing -- #
print('Euclidean Distances to 2017 Bugatti Veyron (V1)')
for car in range(len(data_norm_divide)):
    d = euclidean_dist(data_norm_divide.iloc[4,:3].values, 
                       data_norm_divide.iloc[car, :3].values)
    print('  {}: \t{:01.3f}'.format(car,d))

# -- standardized data -- #
print('\nEuclidean Distances to 2017 Bugatti Veyron (V2)')
for car in range(len(data_unitnorm)):
    d = euclidean_dist(data_unitnorm.iloc[4,:3].values, 
                       data_unitnorm.iloc[car, :3].values)
    print('  {}: \t{:01.3f}'.format(car,d))

Euclidean Distances to 2017 Bugatti Veyron (V1)
  0: 	0.758
  1: 	1.500
  2: 	1.412
  3: 	1.343
  4: 	0.000

Euclidean Distances to 2017 Bugatti Veyron (V2)
  0: 	2.305
  1: 	4.363
  2: 	3.796
  3: 	3.562
  4: 	0.000


For the normalized data (with each feature divided by its maximum value), we get the following distances:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Blazer = 0.758

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Odyssey = 1.500

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Civic = 1.412

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Charger = 1.343

For the standardized data (using sklearn's StandardScaler), we get the following distances:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Blazer = 2.305

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Odyssey = 4.363

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Civic = 3.796

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Bugatti - Charger = 3.562

Notice that both techniques yielded the same order of cars from nearest to farthest.
>This is coincidental, and unlikely to happen with larger or more varied data sets.

Since the distance between the Bugatti and Blazer is the smallest, if $k = 1$, we would classify the Bugatti as cool. However, if $k = 4$ there would no longer be a majority, and we would not be able to classify the Bugatti in either category without a tiebreaker protocol.

Generally speaking, *larger values of $k$ reduce noise, but also make the boundaries between classes less distinct*. The best value of $k$ will vary by data set.

# Example kNN 

Next we will see if we can use k-NNA on the Pima diabetes dataset.

## Loading Data

In [6]:
# -- loading dataset -- #
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = po.read_csv(url, names=names)

# -- dropping NaN rows -- #
invalid = ['plas', 'pres', 'skin', 'test', 'mass']

for i in invalid:
    data[i].replace(to_replace=0, value=np.nan, inplace=True)
    
data = data.dropna(axis=0).reset_index(drop=True)

data.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
1,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
2,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
3,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
4,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1


Now, let's clearly define which columns will act as explanatory variables, and which column will be the target value, and split the dataset between your training data and testing data.  Let's try an 80-20 split and use sklearn's [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) method (set random_state = 0 so we get the same output each time).

In [7]:
from sklearn.model_selection import train_test_split
# columns we will use to make predictions with (features!) feel free to play around with these
X_cols = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']

# column that we want to predict
y_col = 'class'

# 80-20 train-test split of datset
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(data[X_cols], data[y_col], test_size=test_size, random_state=0)

# further split X and y of training nto training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=0)

## Normalizing Data

Let's not forget to normalize the data! We'll use sklearn's StandardScaler normalization like we did before to normalize the training **and** testing data.

In [8]:
scaler = StandardScaler()

for i in list(X_train):
    feature_data_train = X_train[i].values.reshape(-1, 1)
    scaler.fit(feature_data_train)
    X_train[i] = scaler.transform(feature_data_train)

for j in list(X_test):
    feature_data_test = X_test[j].values.reshape(-1, 1)
    scaler.fit(feature_data_test)
    X_test[j] = scaler.transform(feature_data_test)
    
for k in list(X_val):
    feature_data_val = X_val[k].values.reshape(-1, 1)
    scaler.fit(feature_data_val)
    X_val[k] = scaler.transform(feature_data_val)

C:\Users\emily\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emily\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emily\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emily\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emily\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted

## Using sklearn's k-NNA

Luckily for us, sklearn has some quick and easy functions for normalizing the data, finding Euclidean distances, training, and testing with [k-NNA](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Try k = 5 to start.

In [9]:
from sklearn.neighbors import KNeighborsClassifier

# creating model with sklearn's KNeighborsClassifier -- after running these cells play around with the parameter n!
knn = KNeighborsClassifier(n_neighbors=5)

# training/fitting a model with training data
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [12]:
from sklearn.metrics import accuracy_score
import time

y_train_pred=knn.predict(X_train)
start = time.time()
predictions_fast = knn.predict(X_val)

print('Took {} seconds'.format(time.time() - start))
print("Training Accuracy is ", accuracy_score(y_train, y_train_pred)*100)
print("Validation Accuracy is ", accuracy_score(y_val,predictions_fast)*100)

Took 0.002027750015258789 seconds
Training Accuracy is  79.2
Validation Accuracy is  80.95238095238095


# Conclusions and Limitations

k-NNA provides a good baseline classifier for machine learning that is conceptually intuitive and easy to implement and train. One major advantage is that, by standardizing input data, k-NNA can combine any number of features regardless of their original distributions.

However, k-NNA can run into problems with more complex data sets:
* With additional dimensions, it can be harder to define meaningful distances.
* The testing phase can slow down significantly with large data sets since each point's distance is measured to every other point.
* "Majority votes" may be skewed if one classification is significantly more common than the others.
* There is no consideration for correlated features.

As a final note, there are other ways to instantiate k-NNA's, and other types of NNA beyond k-Nearest. For example, to solve the large majority problem, we could have used a weighted voting system where nearer neighbors' votes carry more weight.